In [10]:
import pandas as pd
import os

In [11]:
ruta_archivos = [os.path.join('historicos_cubo', ruta) for ruta in os.listdir('historicos_cubo')]
ruta_archivos = sorted(ruta_archivos, key=lambda x: int(x.split('_')[3]))

In [12]:
dfs_cubo = {'_'.join(ruta.split('_')[4: 6]): pd.read_excel(ruta) for ruta in ruta_archivos}

In [13]:
historico_cubo = {}
for fecha_df, df in dfs_cubo.items():
    gasto_indirecto_tavi = df.loc[66, '270-PROCEDIMIENTOS TAVI']
    historico_cubo[fecha_df] = [gasto_indirecto_tavi]

In [14]:
gastos_tavi = pd.DataFrame.from_dict(historico_cubo).transpose().reset_index()
gastos_tavi.columns = ['Fecha', 'Gasto_Indirecto']
gastos_tavi['Fecha'] = pd.to_datetime(gastos_tavi['Fecha'], format='%Y_%m')

In [15]:
ruta_archivos = [os.path.join('historicos_producciones', ruta) for ruta in os.listdir('historicos_producciones')]
ruta_archivos = sorted(ruta_archivos)


In [16]:
dfs_produccion = {ruta.split()[-2]: pd.read_excel(ruta) for ruta in ruta_archivos}

In [22]:
columnas = {"Unnamed: 2": "ENERO",
            "Unnamed: 3": "FEBRERO",
            "Unnamed: 4": "MARZO",
            "Unnamed: 5": "ABRIL",
            "Unnamed: 6": "MAYO",
            "Unnamed: 7": "JUNIO",
            "Unnamed: 8": "JULIO",
            "Unnamed: 9": "AGOSTO",
            "Unnamed: 10": "SEPTIEMBRE",
            "Unnamed: 11": "OCTUBRE",
            "Unnamed: 12": "NOVIEMBRE",
            "Unnamed: 13": "DICIEMBRE"}

In [26]:
dfs_produccion['2021'] = dfs_produccion['2021'].rename(columns=columnas)
dfs_produccion['2022'] = dfs_produccion['2022'].rename(columns=columnas)

In [64]:
tavi_2021 = dfs_produccion['2021'].loc[62]['ENERO':'DICIEMBRE']
tavi_2022 = dfs_produccion['2022'].loc[62]['ENERO':'DICIEMBRE']

cantidad_tavis = pd.concat([tavi_2021, tavi_2022]).reset_index().drop(columns=['index'])

In [70]:
gastos_tavi['Cantidad_TAVIs'] = cantidad_tavis.astype('Int32')
gastos_tavi = gastos_tavi.replace(0, 0)

In [71]:
gastos_tavi['Gasto_Indirecto_promedio'] = gastos_tavi['Gasto_Indirecto'].divide(gastos_tavi['Cantidad_TAVIs'])

In [73]:
gastos_tavi.to_excel('tavi_indirectos_historicos.xlsx', index=False)